Based on a paper of F. Barlat

## Load libs

In [ ]:
%pylab inline
import time
import lib;reload(lib)
from lib import *
import yf;reload(yf)
from yf import *
import func_hard; reload(func_hard)
from func_hard import *
from MP import progress_bar
uet = progress_bar.update_elapsed_time
import multiprocessing
from multiprocessing import Pool
from scipy import optimize

import modules

- Rotation mat

$\bar{R}= 
  \begin{bmatrix}
  \cos{\psi}  &-\sin{\psi} & 0 \\
  \sin{\psi}  & \cos{\psi} & 0 \\
  0  & 0 & 1 
  \end{bmatrix}                                  
$

In [ ]:
def rot(psi):
    psi = psi * np.pi/180.
    r = np.zeros((3,3)); c = np.cos(psi); s = np.sin(psi)    
    r[0,0]= c;  r[0,1]=-s
    r[1,0]= s;  r[1,1]= c
    r[2,2]= 1.
    return r

def rot_tensor(a,psi):
    """
    Arguments
    ---------
    a
    psi (in degree)
    """
    a=np.array(a)
    b=np.zeros((3,3))
    r=rot(psi)
    for i in xrange(3):
        for j in xrange(3):
            b[i,j] = 0.
            for k in xrange(3):
                for l in xrange(3):
                    b[i,j] = b[i,j] + r[i,k] * a[k,l] * r[j,l]
    return b

- Test fitting for a phenomenological hardening curve

In [ ]:
plt.figure(figsize=(3.5,3.))
xdata,ydata=np.loadtxt('/Users/yj/Documents/ExpDat/IFSteel/Bulge/EXP_BULGE_JINKIM.txt').T
hparams,dum=optimize.curve_fit(f=func_swift,xdata=xdata,ydata=ydata)
eps_bar = np.linspace(0,1.,100)
sig_bar = func_swift(eps_bar,*hparams)
plot(eps_bar,sig_bar)
plot(xdata[::20],ydata[::20],'rx')
print hparams

ss_temp(gca())

### Demonstration of the three important characterisitics
 1. Yield surface and stress loading paths
 2. Corresponding strain loading paths
 3. Equivalent Stress-strain hardening curve

In [ ]:
## Material card
func_har = func_swift
func_ptn = VonMises
func_yld = VonMises ## Associated Flow Rule
eqv_eps  = VonMisesE
##

sigma_bar = lambda eps: func_har(eps,*hparams)
eps_bar   = np.linspace(0,1.5,1000)

#fig=plt.figure(figsize=(3.5,3));ax=gca()
#ax.plot(eps_bar,sigma_bar(eps_bar))
#ax.set_xlabel(r'Strain $\bar{\varepsilon}$')
#ax.set_ylabel(r'Stress $\bar{\sigma}$')

# region A simulations - fairly straight forward.
#d_time = 1e-4 ## time increment.
#sr     = 1e-3 ## major component
delt_ebar = 1e-3 ## incremental step for the hardening curve.
alpha  = 1.0   ## BB
beta   = 0.0   ## No shear

## at the time=0?
eps_mx = 0.05
eps6=np.zeros(6)
ebar = 0.
fig=plt.figure(figsize=(11,3))
ax1=fig.add_subplot(131)
ax2=fig.add_subplot(132)
ax3=fig.add_subplot(133)
i=0
while (ebar<eps_mx):
    ebar      = ebar + delt_ebar
    sig_bar   = sigma_bar(ebar)
    ax3.plot(ebar,sig_bar,'k.')
    delt_work = delt_ebar * sig_bar
    
    ## Current stress tensor "direction"
    sig_norm6 = alph2sig6(alpha,beta)
    sig_norm6 = sig_norm6/func_yld(sig_norm6) ## equivalent stress state
    sig6      = sig_norm6*sig_bar

    ## Current strain rate tensor direction.
    deps6 = alph2eps(alpha,beta,func_ptn)  
    ## Correct deps6 magnitude by conjugating the incremental work
    dw = 0.
    for j in xrange(3):
        dw = dw + deps6[j]*sig6[j]
        dw = dw + 2*deps6[j+3]*sig6[j+3]
    x     = delt_work/dw
    deps6 = deps6*x
    eps6  = eps6 + deps6

    ax1.plot(sig6[0],sig6[1],'k.')
    ax2.plot(eps6[0],eps6[1],'k.')
    
    if i==0: ## draw yield locus
        loc1,loc2=y_locus(100,func_yld)
        loc1=loc1*sig_bar
        ax1.plot(loc1[0],loc1[1])
    i=i+1

ys_temp(ax1)
es_temp(ax2)
ss_temp(ax3)
ax2.grid('on')
ax1.set_xlim(0.,)
ax1.set_ylim(0.,)
fig.tight_layout()
fig.savefig('mk_algorithm.pdf',bbox_inches='tight')

## Numerical procedure

- Summarize what end-user is supposed to specify:

|Symbols                     |Definitions                                          |
|:--------------------------:|:---------------------------------------------------:|
|$\alpha$                    |$\sigma_{11}/\sigma_{22}$                            |
|$\beta$                     |$\sigma_{12}/\sigma_{22}$                            |
|$\dot{\bar{\varepsilon}}$   |Equivalent strain rate                               |
|$\Delta{\bar{\varepsilon}}$ | Incremental step size in terms of equivalent strain |

* Boundary condition
Given value of $\alpha$ and $\beta$ and strain rate determines the boundary condition of region A.

* Description of Strain hardening

The strain-hardening is described as below

$\begin{equation*}
\bar{\sigma}=H(\bar{\varepsilon})
\end{equation*}
$

where $\bar{\sigma}$ and $\bar{\varepsilon}$ are the equivalent stress and strain.

- Isotropic hardening

For the given incremental step size, the corresponding incremental work is useful to follow the isotropic hardening.
To that end, the incremental work is obtained through
$\Delta w = \Delta{\bar{\varepsilon}}\cdot \bar{\sigma}$

Given the *accummulative* equivalent strain tensor  $\varepsilon_{ij}$ where $\bar{\sigma}$ is determined from the strain-hardening

Given a stress path determined by given $\alpha$ and $\beta$,
the stress state can be found through

$\Sigma_{ij} = \bar{\Sigma} \cdot \phi^{yld}(\alpha,\beta)$
where the *direction* of the strain rate vector is determined by the associated flow rule:

$\dot{E}_{k} \propto \frac{\partial\phi^{yld}}{\partial \Sigma_{k}}$

The actual incremental strain *tensor* can be determined based on the equivalent work principle:
$\Delta w = \Delta E_{ij} \bar{\Sigma}_{ij}$

where $\bar{\Sigma}_{ij}$ is determined from the yield surface.

Therefore, $\Delta E_{ij}$ can be found based on $\Delta w / \bar{\Sigma}_{ij}$

For the given incremental step size in terms of $\Delta \bar{E}$, 

the full tensorial strain rate can be determined through following.

The corresponding incrmental 'time' is

$\Delta t = \Delta {\bar{E}}/ \dot{\bar{E}}$

$\dot{\bar{E}}$ is the equivalent strain rate, which is given by the end-user.

$\dot{\bar{E}}_{ij} = \Delta E_{ij}/ \Delta t$

## For region A under a single path ($\alpha$,$\beta$)

In [ ]:
## Functionize the above numerical procedure
def FLD_A_one_path(alpha=1.0,beta=0.,
                   func_yld = VonMises,
                   func_hard = None,
                   ebar_mx=0.1,
                   delt_ebar=1e-4,
                   sr=1e-3,
                   **yld_kwargs):
    """
    Arguments
    ---------
    alpha
    beta
    func_yld
    func_hard
    ebar_mx
    delt_ebar
    sr
    **yld_kwargs
    """
    eps6=np.zeros(6)
    ebar=0.
    time_flow=0.

    strain=[]; stress=[]; strain_eq=[]; 
    stress_eq=[]; time_stamps=[]; strain_rate=[];

    # strain rate
    # sr = delt_ebar / time.
    # Inversely, 
    # time = delt_ebar / sr
    # Therefore, given time, the tensorial strain rate would be...
    # reps6 = deps6 / time

    i=0
    while (ebar<ebar_mx):
        ebar      = ebar + delt_ebar
        sig_bar   = func_hard(ebar) ## static flow stress
        delt_work = delt_ebar * sig_bar

        if i==0: ## when initial anisotropy is considered (constant shape Yield Surface)
            ## Cauchy stress vector direction
            sig_norm6 = alph2sig6(alpha,beta)
            ## equivalent stress state
            sig_norm6 = sig_norm6 / func_yld(sig_norm6,**yld_kwargs)
            ## Strains vector direction (deviatoric strain).
            deps6 = alph2eps(alpha,beta,func_yld,**yld_kwargs)
            pass

        ## Cauchy stress
        sig6        = sig_norm6*sig_bar
        ## Correct deps6 magnitude by conjugating the incremental work
        dw = 0.
        for j in xrange(3):
            dw = dw + deps6[j]*sig6[j]
            dw = dw + 2*deps6[j+3]*sig6[j+3]

        # delta time
        dt          = delt_ebar  / sr
        time_flow   = time_flow  + dt
        x           = delt_work  / dw
        delta_eps6  = deps6      * x
        reps6       = delta_eps6 / dt
        eps6        = eps6       + delta_eps6

        ## stamps
        strain.append(eps6)
        stress.append(sig6)
        strain_eq.append(ebar)
        stress_eq.append(sig_bar)
        strain_rate.append(reps6)
        time_stamps.append(time_flow)        

        i=i+1

    return np.array(strain),np.array(stress),\
           np.array(strain_eq),np.array(stress_eq),\
           np.array(strain_rate), np.array(time_stamps)

# Check different types of yield functions 

Von Mises

In [ ]:
from MP.lib import axes_label

In [ ]:
sigma_bar = lambda eps: func_har(eps,*hparams)
alphs=np.linspace(0.,2.,9)

fig=plt.figure(figsize=(11,6))
ax1=fig.add_subplot(231)
ax2=fig.add_subplot(232)
ax3=fig.add_subplot(233)
ax4=fig.add_subplot(234)
for i in xrange(len(alphs)):
    eps6,sig6,ebar,sbar,sr6,times = FLD_A_one_path(
        alpha=alphs[i],beta=0.,
        func_yld = VonMises,
        func_hard = sigma_bar,
        ebar_mx=0.1,
        delt_ebar=1e-4)

    ax1.plot(eps6[:,1],eps6[:,0],'r-')
    ax2.plot(sig6[:,1],sig6[:,0],'b-')
    ax3.plot(ebar,sbar)
    ax4.plot(times,sr6[:,2])
    
ax2.set_xlim(-50,)
ax2.set_ylim(-50.,)


ax3.set_xlabel(r'$\bar{\epsilon}$',dict(fontsize=15))
ax3.set_ylabel(r'$\bar{\sigma}$',dict(fontsize=15))

ax4.set_xlabel('Time stamps',dict(fontsize=15))
ax4.set_ylabel(r'$\mathrm{\dot{E}_{33}}$',dict(fontsize=17))



ys_tempr(ax2); es_tempr(ax1)
fig.tight_layout()

Hosford

In [ ]:
sigma_bar = lambda eps: func_har(eps,*hparams)
alphs=np.linspace(0.,2.,13)

fig=plt.figure(figsize=(7,6))
ax1=fig.add_subplot(221)
ax2=fig.add_subplot(222)
ax3=fig.add_subplot(223)
for i in xrange(len(alphs)):
    eps6,sig6,ebar,sbar,sr6,times = FLD_A_one_path(
        alpha=alphs[i],beta=0.,
        func_yld = Hosford,
        func_hard = sigma_bar,
        ebar_mx=0.1,
        delt_ebar=1e-4,a=8.)

    ax1.plot(eps6[:,1],eps6[:,0],'r-')
    ax2.plot(sig6[:,1],sig6[:,0],'b-')
    ax3.plot(times,sr6[:,2])
    
ax2.set_xlim(-50,)
ax2.set_ylim(-50.,)
ys_tempr(ax2)
es_tempr(ax1)
fig.tight_layout()

Quadratic Hill

In [ ]:
sigma_bar = lambda eps: func_har(eps,*hparams)
alphs=np.linspace(0.,2.,9)

t0=time.time()
fig=plt.figure(figsize=(11,6))
ax1=fig.add_subplot(231);ax2=fig.add_subplot(232);
ax3=fig.add_subplot(233);ax4=fig.add_subplot(234);

for i in xrange(len(alphs)):
    eps6,sig6,ebar,sbar,sr6,times = FLD_A_one_path(
        alpha = alphs[i],beta=0.,
        func_yld = QuadHill,
        func_hard = sigma_bar,
        ebar_mx=0.05,
        delt_ebar=1e-4,
        R0=0.5, R90=2.0)

    ax1.plot(eps6[:,1],eps6[:,0],'r-')
    ax2.plot(sig6[:,1],sig6[:,0],'b-')
    ax3.plot(ebar,sbar)
    ax4.plot(times,sr6[:,2])

uet(time.time()-t0,head='Elapsed time for Region A probing')
ax2.set_xlim(-50.,); ax2.set_ylim(-50.,)
ys_tempr(ax2);es_tempr(ax1);ss_temp(ax3)
fig.tight_layout()

## Numerical procedure for region B simulations

In the DA approach, the tensorial properties are referred to the Cartensian coordinates with basis vectors $\mathbf{e}^\mathrm{(nt)}$ attached to $\mathbf{n}$ and $\mathbf{t}$ directions.

The numerical procedure begins to look at the *compatibility* between regions A and B. For the given incremental step, the incremental strain for region B is initially unknown. Yet, at least two components can be directly obtained by enforcing normal and tangential components of strain.

When $\Delta \varepsilon^\mathrm{(A)}_{ij}$ is transformed to $\Delta \varepsilon^\mathrm{(A,grv)}_{ij}$, the incremental strain pertaining to region B for the given incremental step should be written as:

$\Delta \varepsilon^{\mathrm{(B,grv)}}_{ij}= 
  \begin{bmatrix}
    x & \Delta \varepsilon^\mathrm{(A,grv)}_{12}  & 0 \\
\Delta \varepsilon^\mathrm{(A,grv)}_{12} & \Delta \varepsilon^\mathrm{(A,grv)}_{22}  & 0 \\
    0 & 0    & -x-\Delta \varepsilon^\mathrm{(A,grv)}_{22}
  \end{bmatrix}
  \mathbf{e}^\mathrm{(nt)}_i \otimes \mathbf{e}^\mathrm{(nt)}_j$

where $x$ is unknown.


Some components of the stress tensor are dictated by the force equilibrium such that

$\bar{\sigma}^{\mathrm{(B,grv)}}_{ij}= \frac{1}{f}
  \begin{bmatrix}
    \bar{\sigma}^\mathrm{(A,grv)}_{11}  & 0        & 0 \\
  0   & \bar{\sigma}^\mathrm{(A,grv)}_{22}  & 0 \\
  0   & 0        & 0 
  \end{bmatrix}                                  
  \mathbf{e}^\mathrm{(nt)}_i \otimes \mathbf{e}^\mathrm{(nt)}_j.  
$

At a time stamp, t, the knowns are: 

$\mathrm{\Delta{\varepsilon_{ij}}^{(A,grv)}, \Delta{\bar{\sigma}^{(A,grv)}}    }$


where $\mathrm{\bar{\sigma}_{ij}^{(B,grv)}}$ is known,

In [ ]:
## Given loading history
eps6,sig6,ebar,sbar,sr6,times ## given information from Region A
func_yld  = QuadHill
func_hard = sigma_bar

## Direction vectors
n    = [1., 0., 0.]; t    = [0., 1., 0.]

## MK params.
f   = 0.990; psi = 0.

## Material cards

t0 = time.time()   ## elapsed time
t  =0.             ## time stamps

eps6_b=np.zeros(6)
for i in xrange(len(eps6)):
    eps6_a = eps6[i,:]; sig6_a = sig6[i,:]
    ebar_a = ebar[i];   sbar_a = sbar[i]
    sr6_a  = sr6[i,:]; ## strain-rate

    ## time increment
    dt     = times[i] - t

    ## Time stamps
    t = t + dt

    """
    ## force equ
    sig_nn_b = sig_nn_a / f
    sig_nt_b = sig_nt_a / f
    ## Compatibility
    eps_nt_b_dot = eps_nt_a_dot
    eps_tt_b_dot = eps_tt_a_dot
    """
    sig33_a = s62c(sig6_a)
    sig33_a_grv = rot_tensor(sig33_a,psi)
    sr33_a = s62c(sr6_a)
    sr33_a_grv = rot_tensor(sr33_a,psi)

    ## Force equilibrium
    sig33_b_grv = np.zeros((3,3))
    sig33_b_grv = sig33_a_grv/f
    
    ## what to be determined:
    ## strain increment 
    ### compatibility
    sr33_b_grv = np.zeros((3,3))
    sr33_b_grv[0,0]=0. ## unknown
    sr33_b_grv[1,1]=sr33_a_grv[1,1]
    sr33_b_grv[2,2]=0. ## unknown
    sr33_b_grv[0,1]=sr33_a_grv[0,1]
    sr33_b_grv[1,0]=sr33_a_grv[1,0]

    sr6_b_grv = c2s6(sr33_b_g)
    
    
    
    ## inhomogenity evolution
    ## Band rotation

uet(time.time()-t0,head='Elapsed time for FLDB calculation')